# MinIO - Uitgebreide Uitleg en Voorbeelden

In dit notebook behandelen we het gebruik van **MinIO**, een object storage service vergelijkbaar met AWS S3. We bekijken hoe we MinIO kunnen gebruiken via:

- CLI (Command Line Interface)
- Python (via `minio` library)
- Web UI

We behandelen de volgende onderwerpen:

1. Connecteren met MinIO (CLI en Python)
2. Bestanden schrijven naar MinIO (CLI en Python)
3. Bestanden lezen van MinIO (Web UI, CLI, Python)
4. Bestanden aanpassen in MinIO (rechten en instellingen)
5. MinIO instellingen aanpassen (CLI en Python)


## 1. Connecteren met MinIO


### 1.1 MinIO CLI

Op de cluster is de MinIO cluster geinstalleerd waarvan je hier een guide kan vinden: https://docs.min.io/docs/minio-client-quickstart-guide.html

In de shell waarin je cli-commando's wil gebruiken moet je eerst dit doen (na het sluiten van de shell instantie is het alias gereset):

In [ ]:
!mc alias set minio_cli http://minio1:9000 bigdata bigdata123

- `minio_cli` is de alias voor de server
- `http://minio1:9000` is het endpoint (in dit geval 1 van de nodes van de cluster)
- `bigdata`/`bigdata123` zijn de access key en secret key

Of het correct gebeurd is kan je als volgt controleren

In [ ]:
!mc alias list

### 1.2 Python (MinIO SDK)

Verbind met de server door gebruik te maken van het minio package:

In [ ]:
from minio import Minio

client = Minio(
    endpoint="minio1:9000",
    access_key="bigdata",
    secret_key="bigdata123",
    secure=False,
    region="eu-west-1"
)

# Check verbinding
try:
    print(client.list_buckets())
    print("Verbonden met MinIO")
except Exception as e:
    print("Fout bij verbinden:", e)

## 2. Bestanden schrijven naar MinIO


### 2.1 CLI

Maak een bucket aan:

In [ ]:
!mc mb minio_cli/mijnclibucket

Upload een bestand:

In [ ]:
!mc cp ulysses.txt minio_cli/mijnclibucket/

Surf nu naar de WebUI van MinIO (http://localhost:9001), login met bovenstaande credentials en bekijk de bucket. Zie je het bestand staan?

### 2.2 Python

Dezelfde stappen kunnen we ook doen in een python script als volgt:

In [ ]:
# Maak een bucket aan
bucket_name = "mijnpythonbucket"
if not client.bucket_exists(bucket_name):
    client.make_bucket(bucket_name)

# Upload bestand
client.fput_object(bucket_name, "davinci_notebooks.txt", "davinci_notebooks.txt")
print("Bestand geüpload naar MinIO")

Ga opnieuw naar de webUI en bekijk of het correct uitgevoerd is

## 3. Bestanden lezen van MinIO


### 3.1 Via Web UI

- Open je browser en ga naar `http://localhost:9000`
- Login met access key / secret key
- Je kunt bestanden downloaden of bekijken door op het bucket te klikken.


### 3.2 Via CLI

Oplijsten van bestanden kan via het ls-commando:

In [ ]:
# Lijst bestanden
!mc ls minio_cli/mijnclibucket

Downloaden van bestanden kan als volgt

In [ ]:
#Download bestand
!mc cp minio_cli/mijnclibucket/ulysses.txt ulysses_downloaded.txt

### 3.3 Via Python

In [ ]:
# Download bestand
client.fget_object(bucket_name, "davinci_notebooks.txt", "notebooks.txt")
print("Bestand gedownload")

# Lijst bestanden
for obj in client.list_objects(bucket_name):
    print(obj.object_name)

## 4. Bestanden aanpassen in MinIO

### 🔒 Private (default)

* Enkel geauthenticeerde gebruikers kunnen het bestand zien
  * Andere krijgen een error bij het gebruiken van de url van het bestand

### 🌍 Public

* Iedereen kan het zien en downloaden.
  * Bvb via de link:
```
http://<minio-server>:9000/mybucket/image.jpg
```
* Use case: hosten statische bestanden, datasets for studenten, publieke assets voor web pagina's.

Het is mogelijk om deze rechten op bestandsniveau toe te passen maar de standaard is om het op bucketniveau te doen.

### 4.1 Rechten aanpassen (CLI)

In [ ]:
# Maak bucket public
!mc policy set public minio_cli/mijnbucket

# Maak bucket private
!mc policy set private minio_cli/mijnbucket

### 4.2 Rechten aanpassen (Python)

* Principal: * → De policy gaat over iedereen, dus ook niet geauthenticeerde gebruikers.
* Action: Welke acties die vastgelegd worden in deze policy
* Resource: Over welke bucket/files het gaat

In [ ]:
# Stel een policy in (voorbeeld: readonly)
policy_readonly = f'''{{
  "Version":"2012-10-17",
  "Statement":[
    {{
      "Effect":"Allow",
      "Principal":{{"AWS":["*"]}},
      "Action":[
        "s3:GetBucketLocation",
        "s3:ListBucket",
        "s3:GetObject"
      ],
      "Resource":[
        "arn:aws:s3:::{bucket_name}",
        "arn:aws:s3:::{bucket_name}/*"
      ]
    }}
  ]
}}'''

client.set_bucket_policy(bucket_name, policy_readonly)
print("Bucket policy aangepast")

## 5. MinIO instellingen aanpassen

MinIO heeft 2 grote categorieën instellingen:

1. **Server configuratie** (zoals credentials, storage class, logging, region, WORM, etc.)
2. **Gebruikers en groepen** (accounts, rechten, policies).

## 🔧 1. Wat kan je aanpassen?

Via `mc admin config` en `mc admin service` kan je o.a.:

* **Gebruikersbeheer**

  * Toevoegen, aanpassen, verwijderen van gebruikers.
  * Wachtwoorden wijzigen.
  * Gebruikers activeren/deactiveren.

* **Region**

  * `region.name` → instellen van de regio van je MinIO cluster.

* **Credentials**

  * `root user` en `root password` aanpassen.

* **Storage class**

  * `storage_class.standard` en `storage_class.rrs` (Reduced Redundancy Storage) configureren.

* **WORM (Write Once Read Many)**

  * `worm=on` om data immutable te maken.

* **Logging**

  * HTTP en audit logging in- of uitschakelen.

* **KMS (Key Management System)**

  * Encryptie instellen.

* **Healing / Erasure coding**

  * Parameters aanpassen voor dataherstel in distributed setups.


### 5.1 Via CLI

**Oefening:** Ga op zoek naar wat de volgende commando's doen

In [ ]:
!mc admin user add minio_cli newuser newpassword
!mc admin user enable minio_cli newuser
!mc admin user list minio_cli

!mc admin config set minio_cli identity_root_user=newroot identity_root_password=newpass123
!mc admin config set minio_cli region name="eu-west-1"
!mc admin config set minio_cli logger_http enabled=on
!mc admin config set minio_cli worm=on
!mc admin service restart minio_cli

!mc admin info myminio

### 5.2 Via Python

De MinIO Python SDK ondersteunt beperkte admin functies. Voor volledige admin functies moet je meestal CLI of de Web UI gebruiken.

Voor beperkte admin acties via Python kun je gebruik maken van `subprocess` om CLI commands aan te roepen:

```python
import subprocess

# Voorbeeld: lijst buckets via CLI
result = subprocess.run(["mc", "ls", "minio_cli"], capture_output=True, text=True)
print(result.stdout)
```

### Extra tips:
- MinIO ondersteunt AWS S3 API, dus je kunt ook `boto3` gebruiken voor interacties.
- Voor productie gebruik altijd `secure=True` en stel TLS in.
- Policies en bucket instellingen kunnen gedetailleerd worden aangepast via JSON policies.

## Oefeningen

Los de volgende oefeningen op, voor sommige oefeningen ga je de nodige functies moeten opzoeken in de documentatie:
* Oefening 1: Metadata en informatie
  * Zoek op in de documentatie welke functie de metadata van een object kan ophalen (hint: stat_object).
  * Gebruik dit om de grootte en content-type van je bestand te tonen.
* Oefening 2: Upload drie bestanden naar een bucket.
  * Verwijder één bestand met remove_object.
  * Controleer met list_objects() of hij effectief weg is.
* Oefening 3: Maak een Python-script dat het volgende doet:
  * Bucket check
    * Maak een bucket aan met een zelfgekozen naam.
    * Als de bucket al bestaat:
    * Leeg de bucket → verwijder alle bestanden, behalve 2 specifieke bestandsnamen (bijv. keep1.txt en keep2.txt).
  * Uploads
    * Controleer of keep1.txt en keep2.txt in de bucket aanwezig zijn.
    * Als een bestand ontbreekt: upload het vanuit de lokale map.


In [ ]:
# Oefening 1
# Eerst een bestand uploaden (als dat nog niet in de bucket zit)

In [ ]:
# Oefening 2
# Upload drie bestanden